# 主程式

In [3]:
import pandas as pd
import numpy as np
import sqlite3

def 連接資料庫(db):
    # 連接到資料庫【月】 =================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()
    sql = '''select * from revenue t'''
    dfRevenue = pd.read_sql(sql,conn)

    # 連接到資料庫【季】
    sql2 = '''select * from financialStatements t'''
    dfFin = pd.read_sql(sql2,conn)

    # 連接到資料庫【年】
    sql3 = '''select * from dividend t'''
    dfDividend = pd.read_sql(sql3,conn) 

    # 連接到資料庫【上市上櫃】
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    ids = dfStock['code']
    # ==================================================================================
    conn.close()
    
    return dfRevenue,dfFin,dfDividend,ids


def 季財報統計(dfFin, stocks):
    
    dfFin2 = dfFin.copy()
    # loc可以使用文字，iloc則是使用數字
    dfFin2 = dfFin2.loc[:,['代碼','營業利益率','每股稅後盈餘(元)','稅後淨利年成長率','營業利益年成長率','季度']]

    # 【季財報資料】
    dataExport2 = pd.DataFrame(columns = ['代碼','營業利益率','盈比','盈4','盈單','稅後淨利年成長率','營業利益年成長率','季度','每股稅後盈餘(元)'])

    for id in stocks:
        dfFin2mask = dfFin2['代碼'] == id
        dfFin3 = dfFin2[dfFin2mask]
        dfFin3 = dfFin3.sort_values(by='季度', ascending=False) # 重整排名

        盈餘單 = []
        盈餘4 = []

        for y in range(len(dfFin3['季度'])):
            期別期 = dfFin3.iloc[y]['季度'][4:]

            if 期別期 == 'Q1':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+1]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q2':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+2]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q3':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+3]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q4':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            
        try:
            dfFin3['盈單'] = 盈餘單
        except:
            dfFin3['盈單'] = 0

        try:
            dfFin3['盈4'] = 盈餘4
        except:
            dfFin3['盈4'] = 0

        dfFin3['盈比'] = dfFin3['盈單']*4/dfFin3['盈4']
        dfFin3.replace([np.inf, -np.inf], 0, inplace=True)
        dataExport2 = pd.concat([dataExport2,dfFin3],axis=0,ignore_index = True)
    dataExport2a = dataExport2.iloc[:,:8]
    
    return dataExport2a

if __name__ == "__main__":
    dfRevenue,dfFin,dfDividend,ids = 連接資料庫('goodinfoRevenue.db')
    dataExport2a = 季財報統計(dfFin, ids)

# 修復重復資料

In [33]:
dfFin2 = dfFin.copy()
# # loc可以使用文字，iloc則是使用數字
# dfFin2 = dfFin2.loc[:,['代碼','營業利益率','每股稅後盈餘(元)','稅後淨利年成長率','營業利益年成長率','季度']]
dfFin2

,代碼,季度,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,稅後淨利率(母公司),每股稅前盈餘(元),每股稅後盈餘(元),每股淨值(元),...,每股投資現金流量(元),每股融資現金流量(元),每股淨現金流量(元),每股自由現金流量(元),負債對淨值比率,長期資金適合率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分(100為滿分)
0,1101,2022Q4,8.41,1.02,5.83,3.65,4.74,0.91,0.74,29.64,...,-2.27,0.83,-0.31,-1.43,93.39,307.9,37.45,4.81,82.51,45.0
1,1101,2022Q3,5.55,-1.48,3.58,1.96,3.68,0.37,0.38,29.05,...,-1.44,1.1,0.55,-1.03,106.90,310.1,45.28,5.06,141.30,40.0
2,1101,2022Q2,4.38,-3.74,1.53,-0.17,2.92,0.08,0.16,31.71,...,-0.07,0.77,1.07,0.15,106.60,310.5,111.4,5.27,344.10,34.0
3,1101,2022Q1,8.69,0.9,5.1,3.73,5.24,0.19,0.20,34.01,...,0.11,0.31,1.15,0.49,92.08,315.6,26.93,4.2,82.29,45.0
4,1101,2021Q4,24.9,18.48,24.36,19.81,18.92,4.25,3.30,32.37,...,-2.17,5.6,6.47,0.93,95.73,329.8,22.74,5.88,24.13,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42103,3430,2021Q4,23.12,11.79,11.6,8.74,8.74,5.84,4.40,22.45,...,-0.83,-8.36,0.93,9.17,81.63,205.3,24.71,-0.19,-1.63,51.0
42104,3430,2021Q3,22.62,11.31,11.03,8.29,8.29,4.11,3.09,-,...,-0.51,-6.08,1.06,7.15,95.16,189.3,24.82,-0.28,-2.55,51.0
42105,3430,2021Q2,22.46,11.57,11.16,8.38,8.38,2.85,2.14,20.54,...,-0.34,-4.13,0.53,4.71,102.50,179.1,24.96,-0.41,-3.67,47.0
42106,3430,2021Q1,22.26,11.1,10.41,7.81,7.81,1.25,0.94,-,...,-0.2,-1.2,0.63,1.83,108.30,173.5,24.98,-0.7,-6.70,43.0


In [42]:
dfConcat = pd.DataFrame(columns = ['代碼', '季度', '營業毛利率', '營業利益率', '稅前淨利率', '稅後淨利率', '稅後淨利率(母公司)',
       '每股稅前盈餘(元)', '每股稅後盈餘(元)', '每股淨值(元)','股東權益報酬率(季累計)', '股東權益報酬率(年預估)', '資產報酬率(季累計)', '資產報酬率(年預估)',
       '營收年成長率', '毛利年成長率', '營業利益年成長率', '稅前淨利年成長率', '稅後淨利年成長率', '稅後淨利年成長率(母公司)',
       '每股稅後盈餘年成長率', '現金(%)', '應收帳款(%)', '存貨(%)', '速動資產(%)', '流動資產(%)',
       '基金與投資(%)', '固定資產(%)', '無形資產(%)', '其他資產(%)', '現金季成長率', '應收帳款季成長率',
       '存貨季成長率', '流動資產季成長率', '基金與投資季成長率', '固定資產季成長率', '無形資產季成長率', '其他資產季成長率',
       '資產總額季成長率', '現金年成長率', '應收帳款年成長率', '存貨年成長率', '流動資產年成長率', '基金與投資年成長率',
       '固定資產年成長率', '無形資產年成長率', '其他資產年成長率', '資產總額年成長率', '應付帳款(%)', '流動負債(%)',
       '長期負債(%)', '其他負債(%)', '負債總額(%)', '普通股股本(%)', '股東權益總額(%)', '應付帳款季成長率',
       '流動負債季成長率', '長期負債季成長率', '其他負債季成長率', '負債總額季成長率', '普通股股本季成長率',
       '股東權益總額季成長率', '應付帳款年成長率', '流動負債年成長率', '長期負債年成長率', '其他負債年成長率',
       '負債總額年成長率', '普通股股本年成長率', '股東權益總額年成長率', '現金比', '速動比', '流動比', '利息保障倍數',
       '現金流量比(季累計)', '現金流量比(年預估)', '營業成本率', '營業費用率', '應收帳款週轉率(次/年)',
       '應收款項收現日數(日)', '應付帳款週轉率(次/年)', '應付款項付現日數(日)', '存貨週轉率(次/年)', '平均售貨日數(日)',
       '固定資產週轉率(次/年)', '總資產週轉率(次/年)', '淨值週轉率(次/年)', '應收帳款佔營收比率(季累計)',
       '應收帳款佔營收比率(年預估)', '存貨佔營收比率(季累計)', '存貨佔營收比率(年預估)', '每股營業現金流量(元)',
       '每股投資現金流量(元)', '每股融資現金流量(元)', '每股淨現金流量(元)', '每股自由現金流量(元)', '負債對淨值比率',
       '長期資金適合率', '所得稅佔稅前淨利比率', '業外損益佔營收比率', '業外損益佔稅前淨利比率', '財報評分(100為滿分)'])

for id in ids:
    dfFin3 = dfFin2[dfFin2['代碼'] == id]
    dfFinNew = pd.DataFrame(columns = ['代碼', '季度', '營業毛利率', '營業利益率', '稅前淨利率', '稅後淨利率', '稅後淨利率(母公司)',
       '每股稅前盈餘(元)', '每股稅後盈餘(元)', '每股淨值(元)','股東權益報酬率(季累計)', '股東權益報酬率(年預估)', '資產報酬率(季累計)', '資產報酬率(年預估)',
       '營收年成長率', '毛利年成長率', '營業利益年成長率', '稅前淨利年成長率', '稅後淨利年成長率', '稅後淨利年成長率(母公司)',
       '每股稅後盈餘年成長率', '現金(%)', '應收帳款(%)', '存貨(%)', '速動資產(%)', '流動資產(%)',
       '基金與投資(%)', '固定資產(%)', '無形資產(%)', '其他資產(%)', '現金季成長率', '應收帳款季成長率',
       '存貨季成長率', '流動資產季成長率', '基金與投資季成長率', '固定資產季成長率', '無形資產季成長率', '其他資產季成長率',
       '資產總額季成長率', '現金年成長率', '應收帳款年成長率', '存貨年成長率', '流動資產年成長率', '基金與投資年成長率',
       '固定資產年成長率', '無形資產年成長率', '其他資產年成長率', '資產總額年成長率', '應付帳款(%)', '流動負債(%)',
       '長期負債(%)', '其他負債(%)', '負債總額(%)', '普通股股本(%)', '股東權益總額(%)', '應付帳款季成長率',
       '流動負債季成長率', '長期負債季成長率', '其他負債季成長率', '負債總額季成長率', '普通股股本季成長率',
       '股東權益總額季成長率', '應付帳款年成長率', '流動負債年成長率', '長期負債年成長率', '其他負債年成長率',
       '負債總額年成長率', '普通股股本年成長率', '股東權益總額年成長率', '現金比', '速動比', '流動比', '利息保障倍數',
       '現金流量比(季累計)', '現金流量比(年預估)', '營業成本率', '營業費用率', '應收帳款週轉率(次/年)',
       '應收款項收現日數(日)', '應付帳款週轉率(次/年)', '應付款項付現日數(日)', '存貨週轉率(次/年)', '平均售貨日數(日)',
       '固定資產週轉率(次/年)', '總資產週轉率(次/年)', '淨值週轉率(次/年)', '應收帳款佔營收比率(季累計)',
       '應收帳款佔營收比率(年預估)', '存貨佔營收比率(季累計)', '存貨佔營收比率(年預估)', '每股營業現金流量(元)',
       '每股投資現金流量(元)', '每股融資現金流量(元)', '每股淨現金流量(元)', '每股自由現金流量(元)', '負債對淨值比率',
       '長期資金適合率', '所得稅佔稅前淨利比率', '業外損益佔營收比率', '業外損益佔稅前淨利比率', '財報評分(100為滿分)'])
    
    for i in range(len(dfFin3)):
        quarter = dfFin3.iloc[i]['季度']
        
        if quarter in dfFinNew['季度'].values:
            continue
        
        dfFinNew = dfFinNew.append(dfFin3.iloc[i], ignore_index=True)
        dfFinNew = dfFinNew.sort_values('季度', ascending=False, ignore_index=True)
    
    dfConcat = dfConcat.append(dfFinNew, ignore_index=True)
    
dfConcat
    

C:\Users\PAN\AppData\Local\Temp\ipykernel_17960\1563035710.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfFinNew = dfFinNew.append(dfFin3.iloc[i], ignore_index=True)
C:\Users\PAN\AppData\Local\Temp\ipykernel_17960\1563035710.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfConcat = dfConcat.append(dfFinNew, ignore_index=True)
C:\Users\PAN\AppData\Local\Temp\ipykernel_17960\1563035710.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfFinNew = dfFinNew.append(dfFin3.iloc[i], ignore_index=True)
C:\Users\PAN\AppData\Local\Temp\ipykernel_17960\1563035710.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfConcat = dfCon

,代碼,季度,營業毛利率,營業利益率,稅前淨利率,稅後淨利率,稅後淨利率(母公司),每股稅前盈餘(元),每股稅後盈餘(元),每股淨值(元),...,每股投資現金流量(元),每股融資現金流量(元),每股淨現金流量(元),每股自由現金流量(元),負債對淨值比率,長期資金適合率,所得稅佔稅前淨利比率,業外損益佔營收比率,業外損益佔稅前淨利比率,財報評分(100為滿分)
0,1101,2023Q1,9.39,0.63,5.63,3.82,5.35,0.21,0.20,30.02,...,-2.61,-0.97,-2.54,-1.48,92.29,293.6,32.09,5.0,88.79,45.0
1,1101,2022Q4,8.41,1.02,5.83,3.65,4.74,0.91,0.74,29.64,...,-2.27,0.83,-0.31,-1.43,93.39,307.9,37.45,4.81,82.51,45.0
2,1101,2022Q3,5.55,-1.48,3.58,1.96,3.68,0.37,0.38,29.05,...,-1.44,1.1,0.55,-1.03,106.90,310.1,45.28,5.06,141.30,40.0
3,1101,2022Q2,4.38,-3.74,1.53,-0.17,2.92,0.08,0.16,31.71,...,-0.07,0.77,1.07,0.15,106.60,310.5,111.4,5.27,344.10,34.0
4,1101,2022Q1,8.69,0.9,5.1,3.73,5.24,0.19,0.20,34.01,...,0.11,0.31,1.15,0.49,92.08,315.6,26.93,4.2,82.29,45.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19254,3430,2021Q3,22.62,11.31,11.03,8.29,8.29,4.11,3.09,-,...,-0.51,-6.08,1.06,7.15,95.16,189.3,24.82,-0.28,-2.55,51.0
19255,3430,2021Q2,22.46,11.57,11.16,8.38,8.38,2.85,2.14,20.54,...,-0.34,-4.13,0.53,4.71,102.50,179.1,24.96,-0.41,-3.67,47.0
19256,3430,2021Q1,22.26,11.1,10.41,7.81,7.81,1.25,0.94,-,...,-0.2,-1.2,0.63,1.83,108.30,173.5,24.98,-0.7,-6.70,43.0
19257,3430,2020Q4,16.24,5.67,4.94,3.6,3.6,1.96,1.43,18.96,...,-1.31,-4.47,0.08,4.58,113.20,164.7,27.08,-0.73,-14.80,34.0


In [ ]:
count = 0

def 連接到資料庫(db):
    global conn,cursor
    # 連接到資料庫A ====================================================================
    conn = sqlite3.connect(db)
    cursor = conn.cursor()

    sql = '''select * from financialStatements t'''
    dfdatabase = pd.read_sql(sql,conn)

    # 連接到資料庫B
    sqlStock = '''select * from stock t'''
    dfStock = pd.read_sql(sqlStock,conn)
    dfStock = pd.DataFrame(dfStock)
    ids = dfStock['code'][count:]
    # ===================================================================================
    return dfdatabase, ids
    
def 導入資料庫(dataExport2):
    for index, row in dataExport2.iterrows():
        try:
            cursor.execute('INSERT INTO financialStatements values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)', row)
            conn.commit()
        except:
            pass

連接到資料庫('goodinfoRevenue.db')
導入資料庫(dfConcat)
conn.close()

In [ ]:
def 季財報統計(dfFin, stocks):
    
    dfFin2 = dfFin.copy()
    # loc可以使用文字，iloc則是使用數字
    dfFin2 = dfFin2.loc[:,['代碼','營業利益率','每股稅後盈餘(元)','稅後淨利年成長率','營業利益年成長率','季度']]

    # 【季財報資料】
    dataExport2 = pd.DataFrame(columns = ['代碼','營業利益率','盈比','盈4','盈單','稅後淨利年成長率','營業利益年成長率','季度','每股稅後盈餘(元)'])

    for id in stocks:
        dfFin2mask = dfFin2['代碼'] == id
        dfFin3 = dfFin2[dfFin2mask]
        dfFin3 = dfFin3.sort_values(by='季度', ascending=False) # 重整排名

        盈餘單 = []
        盈餘4 = []

        for y in range(len(dfFin3['季度'])):
            期別期 = dfFin3.iloc[y]['季度'][4:]

            if 期別期 == 'Q1':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+1]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q2':
                # 盈單 =================================================
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+2]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q3':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)'] + dfFin3.iloc[y+3]['每股稅後盈餘(元)'] - dfFin3.iloc[y+4]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            elif 期別期 == 'Q4':
                # 盈單
                try:
                    盈餘 = dfFin3.iloc[y]['每股稅後盈餘(元)'] - dfFin3.iloc[y+1]['每股稅後盈餘(元)']
                    盈餘單.append(盈餘)
                except:
                    盈餘單.append(0)

                # 盈4
                try:
                    盈4 = dfFin3.iloc[y]['每股稅後盈餘(元)']
                    盈餘4.append(盈4)
                except:
                    盈餘4.append(0)
            
        try:
            dfFin3['盈單'] = 盈餘單
        except:
            dfFin3['盈單'] = 0

        try:
            dfFin3['盈4'] = 盈餘4
        except:
            dfFin3['盈4'] = 0

        dfFin3['盈比'] = dfFin3['盈單']*4/dfFin3['盈4']
        dfFin3.replace([np.inf, -np.inf], 0, inplace=True)
        dataExport2 = pd.concat([dataExport2,dfFin3],axis=0,ignore_index = True)
    dataExport2a = dataExport2.iloc[:,:8]
    
    return dataExport2a